# LangchainDriven Web scraping with OpenAI


In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings  
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import numpy as np

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Load environment variables and Initialize Openai

In [2]:

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

openai = OpenAI(api_key=OPENAI_API_KEY)

C:\Users\amang\AppData\Local\Temp\ipykernel_19308\3873068739.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  openai = OpenAI(api_key=OPENAI_API_KEY)


 Function to scrape the website

In [3]:


def scrape_website(url):
    try:
        loader = WebBaseLoader(url)
        documents = loader.load()
        if documents:
            content = " ".join([doc.page_content for doc in documents])
            return content
        else:
            print("No documents found.")
            return None
    except Exception as e:
        print(f"An error occurred while scraping the website: {e}")
        return None


 Functions: Divide Text into Chunks, Convert Text Chunks into Vector Embeddings, and Store Vectors in a Vector Store Database


In [4]:


def chunk_text(text, chunk_size=500):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks


def text_to_vectors(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectors = embeddings.embed_documents(chunks)  
    return vectors, embeddings  


def store_vectors(chunks, embeddings):
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store

## create a retrieval QA chain

In [5]:

def create_retrieval_qa_chain(vector_store):
    retrieval_qa = RetrievalQA.from_chain_type(
        llm=openai, 
        chain_type="stuff",  
        retriever=vector_store.as_retriever()
    )
    return retrieval_qa

## Function to get response from LLM

In [6]:

def get_llm_response(query, retrieval_qa):
    response = retrieval_qa.invoke(query)
    return response

## Main execution

Entered website URL to scrape: https://en.wikipedia.org/wiki/Artificial_intelligence
Entered querry: what is AI?

In [7]:

if __name__ == "__main__":
    url = input("Enter the website URL to scrape: ")  # Ask the user for the website URL
    scraped_text = scrape_website(url)

    if scraped_text is None or scraped_text.strip() == "":
        print("Failed to retrieve the website content.")
    else:
        print("Scraped Text:\n", scraped_text[:1000])  # Print the first 1000 characters of the scraped text
        
        chunks = chunk_text(scraped_text)
        print(f"Number of chunks created: {len(chunks)}")  # Debugging print
        vectors, embeddings = text_to_vectors(chunks)  # Get vectors and embeddings
        print(f"Number of vectors created: {len(vectors)}")  # Debugging print
        
        vector_store = store_vectors(chunks, embeddings)  # Pass embeddings for FAISS
        
        query = input("Enter your query: ")
        retrieval_qa = create_retrieval_qa_chain(vector_store)
        
        response = get_llm_response(query, retrieval_qa)
        print("Response:\n", response)


Scraped Text:
 



Artificial intelligence - Wikipedia


























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search















Donate








Appearance
















Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Goals




Toggle Goals subsection





1.1
Reasoning and problem-solving








1.2
Knowledge representation








1.3
Planning and decision-making








1.4
Learning








1.5
Natural language processing








1.6
Perception








1.7
Social intelligence








1.8
General intelligence










2
Techniques




Toggle Techniques subsection






c:\Users\amang\miniconda3\envs\langchainEnv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\amang\miniconda3\envs\langchainEnv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Number of vectors created: 389
Response:
 {'query': 'what is AI?', 'result': ' AI stands for Artificial Intelligence and is a field of research in computer science that aims to develop and study methods and software that enable machines to perceive their environment and use learning and intelligence to achieve defined goals. It involves the ability of systems to synthesize information and make decisions, similar to how biological intelligence functions. '}


In [8]:
response

{'query': 'what is AI?',
 'result': ' AI stands for Artificial Intelligence and is a field of research in computer science that aims to develop and study methods and software that enable machines to perceive their environment and use learning and intelligence to achieve defined goals. It involves the ability of systems to synthesize information and make decisions, similar to how biological intelligence functions. '}